# WEB-BASED CHATBOT TRAINIG

In this Python web-based project with source code, we are going to build a chatbot using deep learning and flask techniques. The chatbot will be trained on the dataset which contains categories (intents), pattern and responses. We use a special artificial neural network (ANN) to classify which category the user’s message belongs to and then we will give a random response from the list of responses.

Since we are going to develop a deep learning based model, we need data to train our model. But we are not going to gather or download any large dataset since this is a simple chatbot. We can just create our own dataset in order to train the model. To create this dataset, we need to understand what are the intents that we are going to train. An “intent” is the intention of the user interacting with a chatbot or the intention behind each message that the chatbot receives from a particular user. According to the domain that you are developing a chatbot solution, these intents may vary from one chatbot solution to another. Therefore it is important to understand the right intents for your chatbot with relevance to the domain that you are going to work with.

## Define Intents

I will define few simple intents and bunch of messages that corresponds to those intents and also map some responses according to each intent category. I will create a JSON file named “intents.json” including these data.

## Data Preparation

First we need to import all the required packages

In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Now we load the json file and extract the required data.

In [18]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('Chat-Bot-app/data/intents.json').read()
intents = json.loads(data_file)

When working with text data, we need to perform various preprocessing on the data before design an ANN model. Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [19]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

We vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number. When we use this class for the text pre-processing task, by default all punctuations will be removed, turning the texts into space-separated sequences of words, and these sequences are then split into lists of tokens. 

Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [20]:
# lemmaztize and lower each word and remove duplicates
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

87 documents
23 classes ['about', 'activity', 'age', 'appreciate', 'carlos', 'complaint', 'exclaim', 'goodbye', 'greeting', 'haha', 'help', 'insult', 'jokes', 'name', 'news', 'nicetty', 'no', 'noanswer', 'options', 'programmer', 'riddle', 'suggest', 'thanks']
106 unique lemmatized words ["'s", '.', 'a', 'about', 'age', 'anyone', 'are', 'ask', 'awesome', 'bad', 'be', 'best', 'bye', 'call', 'can', 'carlos', 'complaint', 'could', 'day', 'designed', 'do', 'doing', 'dumb', 'for', 'funny', 'geovany', 'give', 'good', 'goodbye', 'great', 'haha', 'hand', 'have', 'hay', 'he', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'idiot', 'is', 'it', 'joke', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'made', 'make', 'me', 'mexico', 'name', 'need', 'news', 'next', 'nice', 'no', 'nope', 'offered', 'ok', 'old', 'please', 'programmed', 'provide', 'question', 'raise', 'riddle', 'rofl', 'see', 'service', 'should', 'shut', 'suggest', 'suggestion', 'support', 'talk', 'talking', 'tell', 'thank', 't

texts.pkl – This is a pickle file in which we store the words Python object using Nltk that contains a list of our vocabulary.

labels.pkl – The classes pickle file contains the list of categories(Labels).

In [21]:
pickle.dump(words,open('Chat-Bot-app/data/texts.pkl','wb'))
pickle.dump(classes,open('Chat-Bot-app/data/labels.pkl','wb'))

## Model Training

Let’s define our Neural Network architecture for the proposed model and for that we use the “Sequential” model class of Keras.

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [26]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

Our model architecture looks as follows.

In [24]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 128)               13696     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 64)                8256      
                                                                 
 dropout_17 (Dropout)        (None, 64)                0         
                                                                 
 dense_26 (Dense)            (None, 23)                1495      
                                                                 
Total params: 23,447
Trainable params: 23,447
Non-trainable params: 0
_________________________________________________________________


After training, it is better to save all the required files in order to use it at the inference time. So that we save the trained model, fitted tokenizer object and fitted label encoder object.

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 97% accuracy on our model. Let us save the model as ‘model.h5’.

In [27]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('Chat-Bot-app/data/model.h5', hist)

Epoch 1/200
18/18 [==============================] - 2s 3ms/step - loss: 0.2302 - accuracy: 0.9425
Epoch 2/200
18/18 [==============================] - 0s 3ms/step - loss: 0.3124 - accuracy: 0.9080
Epoch 3/200
18/18 [==============================] - 0s 3ms/step - loss: 0.3146 - accuracy: 0.8736
Epoch 4/200
18/18 [==============================] - 0s 3ms/step - loss: 0.1686 - accuracy: 0.9540
Epoch 5/200
18/18 [==============================] - 0s 3ms/step - loss: 0.1799 - accuracy: 0.9655
Epoch 6/200
18/18 [==============================] - 0s 3ms/step - loss: 0.2424 - accuracy: 0.9425
Epoch 7/200
18/18 [==============================] - 0s 3ms/step - loss: 0.1330 - accuracy: 0.9770
Epoch 8/200
18/18 [==============================] - 0s 3ms/step - loss: 0.3576 - accuracy: 0.8621
Epoch 9/200
18/18 [==============================] - 0s 3ms/step - loss: 0.1492 - accuracy: 0.9540
Epoch 10/200
18/18 [==============================] - 0s 3ms/step - loss: 0.2535 - accuracy: 0.9310
Epoch 11/